# CS224W Final Project: Tutorial on the Augmentation of Graphs in PyG

### Jerry Chan, Jihee Suh, John So

## Installation and Setup

### Install PyG

In [1]:
import torch
torch_version = str(torch.__version__)
if "2.4.0" not in torch_version:
  !pip install torch==2.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
print(torch_version)

2.4.0+cu121


In [ ]:
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric
!pip install ogb

Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html


In [ ]:
import os
import random

import numpy as np
from tqdm import tqdm

import torch
from torch_geometric.nn.models import GraphSAGE
from torch_geometric.loader import NeighborLoader
import torch_geometric.transforms as T
from torch_geometric.utils import to_undirected
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Dataset and Tasks



In [ ]:
dataset = PygNodePropPredDataset(name='ogbn-products', root='./products/')
print(dataset, flush=True)
data = dataset[0]
print(data, flush=True)

split_idx = dataset.get_idx_split()
graph = dataset[0]

#evaluator = Evaluator(name='ogbn-products')

/usr/local/lib/python3.10/dist-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_

PygNodePropPredDataset()
Data(num_nodes=2449029, edge_index=[2, 123718280], x=[2449029, 100], y=[2449029, 1])


### Training and Evaluation Utilities

In [ ]:
### From https://github.com/pyg-team/pytorch_geometric/blob/master/examples/ogbn_train.py

# training process
def train(model, optimizer, dataloader: NeighborLoader, transform=None) -> tuple[torch.Tensor, float]:
    model.train()

    total_loss = total_correct = 0
    for batch in tqdm(dataloader):
        optimizer.zero_grad()
        batch = batch.to(device)
        if transform is not None:
          batch = transform(batch)
        out = model(batch.x, batch.edge_index)[:batch.batch_size]
        y = batch.y[:batch.batch_size].squeeze().to(torch.long)
        loss = torch.nn.functional.cross_entropy(out, y)
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y).sum())
    loss = total_loss / len(train_loader)
    approx_acc = total_correct / split_idx['train'].size(0)
    return loss, approx_acc


@torch.no_grad()
def test(model, dataloader: NeighborLoader, transform=None) -> float:
    model.eval()

    total_correct = total_examples = 0
    total_loss = 0
    for batch in tqdm(dataloader):
        batch = batch.to(device)
        if transform is not None:
          batch = transform(batch)
        out = model(batch.x, batch.edge_index)
        pred = out.argmax(dim=-1)
        y = batch.y.view(-1).to(torch.long)

        loss = torch.nn.functional.cross_entropy(out, y)
        total_loss += float(loss)

        total_correct += int((pred == y).sum())
        total_examples += y.size(0)

    return total_loss / len(dataloader), total_correct / total_examples


In [ ]:
input_dim = dataset[0].x.shape[1]
hidden_dim = 128
learning_rate = 0.0001
num_epochs = 20
batch_size = 32
num_layers = 2

fan_out = 10
num_workers = 2

loss_fn = torch.nn.CrossEntropyLoss()
model = GraphSAGE(in_channels=input_dim, hidden_channels=hidden_dim, num_layers=num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.to(device)
model

GraphSAGE(100, 128, num_layers=2)

In [ ]:
train_loader = NeighborLoader(
    data,
    input_nodes=split_idx['test'],
    num_neighbors=[fan_out] * num_layers,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=num_workers
)
# val_loader = NeighborLoader(
#     data,
#     input_nodes=split_idx['test'],
#     num_neighbors=[fan_out] * num_layers,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=num_workers,
# )
# test_loader = NeighborLoader(
#     data,
#     input_nodes=split_idx['test'],
#     num_neighbors=[fan_out] * num_layers,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=0
# )

/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [ ]:
times = []
best_val = 0.
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train(model, optimizer, train_loader)
    val_loss, val_acc = test(model, train_loader)

    print(f'Epoch {epoch:02d}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc * 100.0:.2f}%',)
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc * 100.0:.2f}%')

    if val_acc > best_val:
        best_val = val_acc

print(f'Best Validation Accuracy: {100.0 * best_val:.2f}%')

print('Testing...')
test_loss, test_final_acc = test(train_loader)
print(f'Test Accuracy: {100.0 * test_final_acc:.2f}%')

  5%|▍         | 3415/69160 [01:04<20:48, 52.65it/s]Exception in thread Thread-13 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 55, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 32, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 496, in rebuild_storage_fd

    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_c

KeyboardInterrupt: 

## Training Data Augmentation

Half-Hop

In [ ]:
halfhop = T.HalfHop(alpha=0.5, p=1.0)

In [ ]:
def train_halfhop(model, optimizer, dataloader: NeighborLoader, transform=None) -> tuple[torch.Tensor, float]:
    model.train()

    total_loss = total_correct = 0
    for batch in tqdm(dataloader):
        optimizer.zero_grad()
        batch = batch.to(device)
        if transform is not None:
          batch = transform(batch)
        out = model(batch.x, batch.edge_index)[~batch.slow_node_mask][:batch.batch_size]
        y = batch.y[:batch.batch_size].squeeze().to(torch.long)
        loss = torch.nn.functional.cross_entropy(out, y)
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y).sum())
    loss = total_loss / len(train_loader)
    approx_acc = total_correct / split_idx['train'].size(0)
    return loss, approx_acc

@torch.no_grad()
def test_halfhop(model, dataloader: NeighborLoader, transform=None) -> float:
    model.eval()

    total_correct = total_examples = 0
    total_loss = 0
    for batch in tqdm(dataloader):
        batch = batch.to(device)
        if transform is not None:
          batch = transform(batch)
        out = model(batch.x, batch.edge_index)[~batch.slow_node_mask]
        pred = out.argmax(dim=-1)
        y = batch.y.view(-1).to(torch.long)

        loss = torch.nn.functional.cross_entropy(out, y)
        total_loss += float(loss)

        total_correct += int((pred == y).sum())
        total_examples += y.size(0)

    return total_loss / len(dataloader), total_correct / total_examples

In [ ]:
model.reset_parameters()

times = []
best_val = 0.
num_epochs = 1
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_halfhop(model, optimizer, train_loader, transform=halfhop)
    val_loss, val_acc = test_halfhop(model, val_loader, transform=halfhop)

    print(f'Epoch {epoch:02d}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc * 100.0:.2f}%',)
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc * 100.0:.2f}%')

    if val_acc > best_val:
        best_val = val_acc

print(f'Best Validation Accuracy: {100.0 * best_val:.2f}%')

print('Testing...')
test_loss, test_final_acc = test_halfhop(model, test_loader, transform=halfhop)
print(f'Test Accuracy: {100.0 * test_final_acc:.2f}%')

100%|██████████| 932/932 [00:10<00:00, 86.70it/s] 


Epoch 01, Train Loss: 2.5389, Train Acc: 34.33%
Val Loss: 2.1922, Val Acc: 39.85%
Best Validation Accuracy: 39.85%
Testing...


100%|██████████| 1519/1519 [00:06<00:00, 247.96it/s]

Test Accuracy: 37.04%


### Mask Feature

In [ ]:
from torch_geometric.utils import mask_feature

def train_with_mask(model, optimizer, dataloader: NeighborLoader, p = 0.2) -> tuple[torch.Tensor, float]:
    model.train()

    total_loss = total_correct = 0
    for batch in tqdm(dataloader):
        optimizer.zero_grad()
        batch = batch.to(device)
        masked_x, feature_mask = mask_feature(batch.x, p)
        out = model(masked_x, batch.edge_index)[:batch.batch_size]
        y = batch.y[:batch.batch_size].squeeze().to(torch.long)
        loss = torch.nn.functional.cross_entropy(out, y)
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y).sum())
    loss = total_loss / len(train_loader)
    approx_acc = total_correct / split_idx['train'].size(0)
    return loss, approx_acc

In [ ]:
times = []
best_val = 0.
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_with_mask(model, optimizer, train_loader)
    val_loss, val_acc = test(model, val_loader)

    print(f'Epoch {epoch:02d}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc * 100.0:.2f}%',)
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc * 100.0:.2f}%')

    if val_acc > best_val:
        best_val = val_acc

print(f'Best Validation Accuracy: {100.0 * best_val:.2f}%')

print('Testing...')
test_loss, test_final_acc = test(test_loader)
print(f'Test Accuracy: {100.0 * test_final_acc:.2f}%')